# TP2 - Détection d'Anomalies (Isolation Forest)

Ce notebook charge les données ECG préparées et applique l'algorithme Isolation Forest pour la détection d'anomalies.

In [ ]:
# --- Imports ---
from pathlib import Path
import sys
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report
)
import warnings

In [3]:
# --- Configuration des Chemins ---

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED_PATH = PROJECT_ROOT / "data" / "ECG"
PROJECT_FCT = PROJECT_ROOT / "src"
sys.path.append(str(PROJECT_FCT))

print(f"Chemin du projet racine : {PROJECT_ROOT}")
print(f"Chargement des données depuis : {DATA_PROCESSED_PATH}")

Chemin du projet racine : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git
Chargement des données depuis : C:\Users\sebdr\OneDrive\Bureau\sherbrooke\IFT 599 - Sciences des données\TP2 Devoir\git\data\ECG


In [5]:
# --- Chargement des modules locaux ---
try:
    print("Modules locaux chargés.")
except ImportError:
    print("Pas de modules locaux spécifiques trouvés dans /src.")

warnings.filterwarnings('ignore')

Modules locaux chargés.


In [6]:
# --- Chargement des données ---

print("Chargement des données ECG préparées (depuis data/processed/)...")
try:
    X_train_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_train_scaled.npy')
    X_val_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_val_scaled.npy')
    X_test_scaled = np.load(DATA_PROCESSED_PATH / 'ecg_X_test_scaled.npy')
    y_train = np.load(DATA_PROCESSED_PATH / 'ecg_y_train.npy')
    y_val = np.load(DATA_PROCESSED_PATH / 'ecg_y_val.npy')
    y_test = np.load(DATA_PROCESSED_PATH / 'ecg_y_test.npy')
    
    print(f"Données d'entraînement (normales): {X_train_scaled.shape}")
    print(f"Données de validation (mixtes): {X_val_scaled.shape}")
    print(f"Données de test (mixtes): {X_test_scaled.shape}")

except FileNotFoundError:
    print(f"ERREUR: Fichiers non trouvés dans {DATA_PROCESSED_PATH}")
    print("Veuillez d'abord exécuter le notebook 'PreparationData.ipynb'.")

Chargement des données ECG préparées (depuis data/processed/)...
Données d'entraînement (normales): (1247, 140)
Données de validation (mixtes): (791, 140)
Données de test (mixtes): (2960, 140)


# 1. Isolation Forest

1.1. Trouver la contamination optimale (sur Validation)

In [7]:
print("Recherche du seuil (contamination) optimal pour Isolation Forest...")
# Tester différents seuils
contaminations = np.linspace(0.01, 0.5, 50)
f1_scores_if = []

for cont in contaminations:
    model_if_val = IsolationForest(contamination=cont, random_state=42)
    model_if_val.fit(X_train_scaled)
    
    # Prédire (1=normal, -1=anomalie)
    y_pred_raw = model_if_val.predict(X_val_scaled)
    # Convertir (0=normal, 1=anomalie)
    y_pred_val = np.where(y_pred_raw == -1, 1, 0)
    
    f1_scores_if.append(f1_score(y_val, y_pred_val, pos_label=1))

CONTAMINATION_OPTIMALE_IF = contaminations[np.argmax(f1_scores_if)]
print(f"Meilleur F1-Score (Val): {np.max(f1_scores_if):.4f}")
print(f"Contamination optimale (Val): {CONTAMINATION_OPTIMALE_IF:.4f}")

Recherche du seuil (contamination) optimal pour Isolation Forest...
Meilleur F1-Score (Val): 0.9749
Contamination optimale (Val): 0.1600


1.2. Entraînement et Évaluation (sur Test)

In [8]:
print("--- Application d'Isolation Forest (Test) ---")
model_if = IsolationForest(contamination=CONTAMINATION_OPTIMALE_IF, random_state=42)
model_if.fit(X_train_scaled)

# Prédire sur le test (1=normal, -1=anomalie)
y_pred_if_raw = model_if.predict(X_test_scaled)
# Convertir en (0=normal, 1=anomalie) pour correspondre à y_test
y_pred_if = np.where(y_pred_if_raw == -1, 1, 0)

# Scores pour ROC-AUC (scores_samples retourne l'opposé du score d'anomalie)
scores_if_test = -model_if.decision_function(X_test_scaled)

--- Application d'Isolation Forest (Test) ---


In [9]:
# --- Métriques de performance (Isolation Forest) ---
accuracy_if = accuracy_score(y_test, y_pred_if)
precision_if = precision_score(y_test, y_pred_if, pos_label=1)
recall_if = recall_score(y_test, y_pred_if, pos_label=1)
f1_if = f1_score(y_test, y_pred_if, pos_label=1)
roc_auc_if = roc_auc_score(y_test, scores_if_test)

print(f"Accuracy: {accuracy_if:.4f}")
print(f"Precision (Anomalie): {precision_if:.4f}")
print(f"Recall (Anomalie): {recall_if:.4f}")
print(f"F1-Score (Anomalie): {f1_if:.4f}")
print(f"ROC-AUC: {roc_auc_if:.4f}")

print("\nRapport de Classification :")
print(classification_report(y_test, y_pred_if, target_names=['Normal (0)', 'Anomalie (1)']))

Accuracy: 0.9622
Precision (Anomalie): 0.9572
Recall (Anomalie): 0.9966
F1-Score (Anomalie): 0.9765
ROC-AUC: 0.9218

Rapport de Classification :
              precision    recall  f1-score   support

  Normal (0)       0.98      0.83      0.90       624
Anomalie (1)       0.96      1.00      0.98      2336

    accuracy                           0.96      2960
   macro avg       0.97      0.91      0.94      2960
weighted avg       0.96      0.96      0.96      2960

